<a href="https://colab.research.google.com/github/nurfnick/Metaheuristics/blob/main/Homework7/SimmulatedAnnealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meta-Heuristics Homework 7 Simmulated Annealing

In [11]:
import copy
from random import Random   #need this for the random number generation -- do not change
import numpy as np


In [12]:
#to setup a random number generator, we will specify a "seed" value
#need this for the random number generation -- do not change
seed = 51132023
myPRNG = Random(seed)

#to get a random number between 0 and 1, use this:             myPRNG.random()
#to get a random number between lwrBnd and upprBnd, use this:  myPRNG.uniform(lwrBnd,upprBnd)
#to get a random integer between lwrBnd and upprBnd, use this: myPRNG.randint(lwrBnd,upprBnd)

#number of elements in a solution
n = 150

#create an "instance" for the knapsack problem
value = []
for i in range(0,n):
    value.append(round(myPRNG.triangular(5,1000,200),1))

weights = []
for i in range(0,n):
    weights.append(round(myPRNG.triangular(10,200,60),1))

#define max weight for the knapsack
maxWeight = 2500


In [13]:
solutionsChecked = 0

#function to evaluate a solution x
def evaluate(x):

    a=np.array(x)
    b=np.array(value)
    c=np.array(weights)

    totalValue = np.dot(a,b)     #compute the value of the knapsack selection
    totalWeight = np.dot(a,c)    #compute the weight value of the knapsack selection

    if totalWeight > maxWeight:
         #print ("Oh no! The solution is infeasible!  What to do?  What to do?")   #you will probably want to change this...
         totalValue = -totalValue

    return [totalValue, totalWeight]   #returns a list of both total value and total weight


#here is a simple function to create a neighborhood
#1-flip neighborhood of solution x
def neighborhood(x):

    nbrhood = []

    for i in range(0,n):
        nbrhood.append(x[:])
        if nbrhood[i][i] == 1:
            nbrhood[i][i] = 0
        else:
            nbrhood[i][i] = 1

    return nbrhood
#take all the 1's and 0's and slide them around, keeping the order.
def neighborhood_slide(x):

    nbrhood = []

    for i in range(0,len(x)):
        newlist = x[i+1:]
        newlist.extend(x[:i+1])
        nbrhood.append(newlist)
    return nbrhood


def neighborhood_slide_and_flip(x):
  nbrhood = []

  for i in neighborhood(x):
    nbrhood.extend(neighborhood_slide(i))

  return nbrhood


#create the initial solution
def initial_solution_zero():
    x = []   #i recommend creating the solution as a list

    for i in range(n):
        x.append(0)

    return x

def initial_solution_random():
  x = []

  for i in range(n):
    x.append(myPRNG.randint(0,1))

  return x

def initial_solution_random_start_feasible():
  x = []

  for i in range(n):
    x.append(myPRNG.randint(0,1))

  i = myPRNG.randint(0,n-1)
  while evaluate(x)[1] > maxWeight:
    x[i] = 0
    i=myPRNG.randint(0,n-1)

  return x
#varaible to record the number of solutions evaluated
solutionsChecked = 0

x_curr = initial_solution_zero()  #x_curr will hold the current solution
x_best = x_curr[:]           #x_best will hold the best solution
f_curr = evaluate(x_curr)    #f_curr will hold the evaluation of the current soluton
f_best = f_curr[:]


## Determine Initial Temp

From the notes:

*If maximal difference in cost between neighboring solutions is
known, initial temperature can be calculated so that increases of
that magnitude are initially accepted with sufficiently large
probability*

For our example, we know that the max value is 1000.  An increase in our value by 1000 would be the best.  We start there for our initial temp.

In [14]:
def probability_assignment(x,T):
  if x>0:
    return 1
  else:
    return np.exp(x/T)

In [27]:
T

0.25572026969116246

In [41]:
#begin local search overall logic ----------------
x_curr = initial_solution_random_start_feasible()  #x_curr will hold the current solution
x_best = x_curr[:]           #x_best will hold the best solution
f_curr = evaluate(x_curr)    #f_curr will hold the evaluation of the current soluton
f_best = f_curr[:]



solutionsChecked = 0
bettersolutionsfound = 0

T = 1000

for i in range(500):

  for i in range(25):
    Neighborhood = neighborhood(x_curr)   #create a list of all neighbors in the neighborhood of x_curr
    probs = []

    for s in Neighborhood:                #evaluate every member in the neighborhood of x_curr
        probs.append(probability_assignment(evaluate(s)[0]-f_curr[0],T))
        #solutionsChecked = solutionsChecked + 1
        #if evaluate(s)[0] > f_curr[0]:#compare to current not best
            #improvements.append(s[:])                 #add to list
            #improvements.append(evaluate(s)[:])       #and store its evaluation
    if sum(probs)<1:
      break
    else:
      x_curr = myPRNG.choices(Neighborhood,probs,k=1)[0]
      f_curr = evaluate(x_curr)
      solutionsChecked += 1

    if f_curr[0]>f_best[0]:
      x_best = x_curr[:]
      f_best = f_curr[:]
      bettersolutionsfound +=1


  T = 0.99*T

print ("\nFinal number of solutions checked: ", solutionsChecked)
print ("Best value found: ", f_curr[0])
print ("Weight is: ", f_curr[1])
print ("Total number of items selected: ", np.sum(x_curr))
print ("Best solution: ", x_curr)


Final number of solutions checked:  5508
Best value found:  23240.8
Weight is:  2478.0
Total number of items selected:  44
Best solution:  [0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1]


In [40]:
T

6.570483042414598

In [26]:
print ("\nFinal number of solutions checked: ", solutionsChecked)
print ("Best value found: ", f_curr[0])
print ("Weight is: ", f_curr[1])
print ("Total number of items selected: ", np.sum(x_curr))
print ("Best solution: ", x_curr)


Final number of solutions checked:  8230
Best value found:  24718.400000000005
Weight is:  2496.9
Total number of items selected:  45
Best solution:  [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1]


In [48]:
#begin local search overall logic ----------------
x_curr = initial_solution_random_start_feasible()  #x_curr will hold the current solution
x_best = x_curr[:]           #x_best will hold the best solution
f_curr = evaluate(x_curr)    #f_curr will hold the evaluation of the current soluton
f_best = f_curr[:]



solutionsChecked = 0
bettersolutionsfound = 0

T = 1500

for i in range(1000):

  for i in range(5):
    Neighborhood = neighborhood(x_curr)   #create a list of all neighbors in the neighborhood of x_curr
    p = 0
    howLongInWhile = 0
    while p ==0:
      solutionsChecked += 1
      howLongInWhile +=1
      apval = myPRNG.random()
      s = Neighborhood[myPRNG.randint(0,len(Neighborhood)-1)]
      if evaluate(s)[0]>f_curr[0]:
        x_curr = s[:]
        f_curr = evaluate(s)[:]
        p =1
      elif probability_assignment(evaluate(s)[0]-f_curr[0],T)>apval :
        x_curr = s[:]
        f_curr = evaluate(s)[:]
        p=1
      elif howLongInWhile >50:
        break

  T = 0.99*T

print ("\nFinal number of solutions checked: ", solutionsChecked)
print ("Best value found: ", f_curr[0])
print ("Weight is: ", f_curr[1])
print ("Total number of items selected: ", np.sum(x_curr))
print ("Best solution: ", x_curr)


Final number of solutions checked:  197642
Best value found:  23030.2
Weight is:  2493.6000000000004
Total number of items selected:  41
Best solution:  [1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0]


In [45]:
print ("\nFinal number of solutions checked: ", solutionsChecked)
print ("Best value found: ", f_curr[0])
print ("Weight is: ", f_curr[1])
print ("Total number of items selected: ", np.sum(x_curr))
print ("Best solution: ", x_curr)


Final number of solutions checked:  1741124
Best value found:  24137.400000000005
Weight is:  2499.9000000000005
Total number of items selected:  44
Best solution:  [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]
